In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

from nvdlib.nvd import NVD

In [2]:
import nltk

In [3]:
import ast
df = pd.read_csv('dataframe-nvd-2017.csv', converters={'version_range': ast.literal_eval})

In [4]:
import plotly

from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

In [5]:
%matplotlib inline

## Prepare the DataFrame

In [6]:
nvd = NVD.from_feeds(['2017'])

In [7]:
nvd.update()

In [8]:
GH_BASE_URL = u"http[s]://github.com"

In [9]:
ecos = ['Java', 'JavaScript', 'Python']
df_ecos = df[ecos]
df_ecos = df[['username', 'project', 'version_range', 'url', *ecos]][df_ecos.any(1)]

df_ecos.set_index(['username', 'project'])

,,version_range,url,Java,JavaScript,Python
username,project,,,,,
Microsoft,ChakraCore,"(None, None)",https://github.com/Microsoft/ChakraCore,0,49258214,84984
torproject,tor,"(0.3.0.7, 0.2.4.28)",https://github.com/torproject/tor,0,0,245996
lota,phamm,"(0.6.6, 0.6.6)",https://github.com/lota/phamm,0,1170,0
JiounDai,CVE-2017-0478,"(None, None)",https://github.com/JiounDai/CVE-2017-0478,6631,0,0
alephsecurity,PoCs,"(None, None)",https://github.com/alephsecurity/PoCs,0,0,1950
zulip,zulip,"(1.4.2, 1.4.2)",https://github.com/zulip/zulip,0,2403124,4774989
recurly,recurly-client-python,"(2.4.0, 2.6.1)",https://github.com/recurly/recurly-client-python,0,0,156458
fedora-infra,fedmsg,"(0.18.1, 0.18.1)",https://github.com/fedora-infra/fedmsg,0,0,385668
twisted,txaws,"(-, -)",https://github.com/twisted/txaws,0,0,783299


In [10]:
import re
def get_reference(cve, url=None, pattern=None) -> str:
    for ref in cve.references:
        if url and url == ref:
            return ref

        if re.search(pattern, ref):
            return ref

    return None

In [11]:
def strip_src_url(url: str):
    """ATM assume that the only reference to source is github."""
    url_base_pattern = u"http[s]://github.com/([\w-]+)/([\w-]+[.]*[\w-]+)"
    strip_url = re.search(url_base_pattern, url)
    
    if not strip_url:
        print(url)
        return None
    
    return strip_url[0]

In [12]:
# Get descriptions and append them to the current DataFrame to avoid recreating a new one from scratch
cves = dict()
cve_list = list()

for cve in nvd.cves():
    ref = get_reference(cve, pattern="http[s]://github.com")
    if ref is None:
        continue
        
    ref = strip_src_url(ref)
    cve_list.append((cve.cve_id, ref, cve.description))
    cves[cve.cve_id] = cve

In [13]:
df_desc = pd.DataFrame(cve_list, columns=['cve_id', 'url', 'description'])

In [14]:
df = pd.merge(df_ecos, df_desc, how='inner', on='url').set_index(['username', 'project'])

In [15]:
df.columns.tolist()
df = df[[
    'cve_id',
    'url',
    'description',
    'version_range',
    'Java',
    'JavaScript',
    'Python',
]]

In [16]:
df

cve_id  \
username            project                                    
Microsoft           ChakraCore                 CVE-2017-0028   
                    ChakraCore                 CVE-2017-0152   
                    ChakraCore                 CVE-2017-0196   
                    ChakraCore                 CVE-2017-0223   
                    ChakraCore                 CVE-2017-0252   
torproject          tor                        CVE-2017-0375   
                    tor                        CVE-2017-0376   
                    tor                        CVE-2017-0377   
                    tor                        CVE-2017-0380   
lota                phamm                      CVE-2017-0378   
JiounDai            CVE-2017-0478              CVE-2017-0478   
alephsecurity       PoCs                       CVE-2017-0563   
zulip               zulip                      CVE-2017-0881   
                    zulip                      CVE-2017-0896   
                    zulip                      CVE-2017-0910   
recurly             recurly-client-python      CVE-2017-0906   
fedora-infra        fedmsg                  CVE-2017-1000001   
twisted             txaws                   CVE-2017-1000007   
akeneo              pim-community-dev       CVE-2017-1000009   
Samnan              MyWebSQL                CVE-2017-1000011   
DSB                 MySQLDumper             CVE-2017-1000012   
ljharb              qs                      CVE-2017-1000048   
xwiki-labs          cryptpad                CVE-2017-1000051   
kubernetes          kubernetes              CVE-2017-1000056   
                    kubernetes              CVE-2017-1002100   
                    kubernetes              CVE-2017-1002101   
                    kubernetes              CVE-2017-1002102   
lsh123              xmlsec                  CVE-2017-1000061   
openmediavault      openmediavault          CVE-2017-1000065   
modxcms             revolution              CVE-2017-1000067   
...                                                      ...   
bhdresh             CVE-2017-8759              CVE-2017-8759   
irods               irods                      CVE-2017-8799   
stoth68000          media-tree                 CVE-2017-8831   
s3131212            allendisk                  CVE-2017-8832   
                    allendisk                  CVE-2017-8848   
                    allendisk                  CVE-2017-9090   
                    allendisk                  CVE-2017-9091   
                    allendisk                  CVE-2017-9249   
                    allendisk                  CVE-2017-9307   
DeuxHuitHuit        symphony-2                 CVE-2017-8876   
mailcow             mailcow-dockerized         CVE-2017-8928   
simpleinvoices      simpleinvoices             CVE-2017-8930   
victorwon           calendarxp                 CVE-2017-9072   
kkos                oniguruma                  CVE-2017-9224   
                    oniguruma                  CVE-2017-9225   
                    oniguruma                  CVE-2017-9226   
                    oniguruma                  CVE-2017-9227   
                    oniguruma                  CVE-2017-9228   
                    oniguruma                  CVE-2017-9229   
tikiorg             tiki                       CVE-2017-9305   
E2OpenPlugins       e2openplugin-OpenWebif     CVE-2017-9333   
                    e2openplugin-OpenWebif     CVE-2017-9807   
BlackCatDevelopment BlackCatCMS                CVE-2017-9609   
eclipse             jetty.project              CVE-2017-9735   
flatpak             flatpak                    CVE-2017-9780   
NancyFx             Nancy                      CVE-2017-9785   
apache              httpd                      CVE-2017-9798   
arvidn              libtorrent                 CVE-2017-9847   
eclipse             mosquitto                  CVE-2017-9868   
memcached           memcached                  CVE-2017-9951   

                     

## Create toy data set

In [17]:
toy_df = df[['cve_id', 'description']][df.Java > 1E2]

In [18]:
# labels are a position of the project token in the token list
labels = [None] * len(toy_df)
for i, (index, row) in enumerate(toy_df.iterrows()):
    proj = index[1]
    desc = row.description.lower()
    # find the position of proj in the description, if applicable
    tokens = nltk.word_tokenize(desc)
    found = False
    for pos, token in enumerate(tokens):
        if token == proj:
            found = True
            break
    if not found:
        pos = None
    labels[i] = (row.cve_id, token, pos)

# turn index into series
labels = pd.DataFrame(labels, columns=['cve_id', 'label', 'pos'])

In [19]:
toy_df = toy_df.reset_index().merge(labels, how='outer', on='cve_id').set_index(['username', 'project'])

In [20]:
toy_df = toy_df[toy_df.pos.notnull()]

In [21]:
toy_df[['description', 'pos']].reset_index().style

,username,project,description,pos
0,modxcms,revolution,MODX Revolution version 2.x - 2.5.6 is vulnerable to blind SQL injection caused by improper sanitization by the escape method resulting in authenticated user accessing database and possibly escalating privileges.,1
1,modxcms,revolution,"In MODX Revolution 2.5.7, the ""key"" and ""name"" parameters in the System Settings module are vulnerable to XSS. A malicious payload sent to connectors/index.php will be triggered by every user, when they visit this module.",2
2,modxcms,revolution,Directory traversal in setup/processors/url_search.php (aka the search page of an unused processor) in MODX Revolution 2.5.7 might allow remote attackers to obtain system directory information.,16
3,modxcms,revolution,"In MODX Revolution before 2.5.7, when PHP 5.3.3 is used, an attacker is able to include and execute arbitrary files on the web server due to insufficient validation of the action parameter to setup/index.php, aka directory traversal.",2
4,modxcms,revolution,"In MODX Revolution before 2.5.7, an attacker is able to trigger Reflected XSS by injecting payloads into several fields on the setup page, as demonstrated by the database_type parameter.",2
5,modxcms,revolution,"In MODX Revolution before 2.5.7, a user with file upload permissions is able to execute arbitrary code by uploading a file with the name .htaccess.",2
6,modxcms,revolution,"In MODX Revolution before 2.5.7, a user with resource edit permissions can inject an XSS payload into the title of any post via the pagetitle parameter to connectors/index.php.",2
7,modxcms,revolution,"In MODX Revolution before 2.5.7, an attacker might be able to trigger XSS by injecting a payload into the HTTP Host header of a request. This is exploitable only in conjunction with other issues such as Cache Poisoning.",2
8,ngallagher,simplexml,"SimpleXML (latest version 2.7.1) is vulnerable to an XXE vulnerability resulting SSRF, information disclosure, DoS and so on.",0
9,swagger-api,swagger-parser,"A vulnerability in Swagger-Parser's version <= 1.0.30 and Swagger codegen version <= 2.2.2 yaml parsing functionality results in arbitrary code being executed when a maliciously crafted yaml Open-API specification is parsed. This in particular, affects the 'generate' and 'validate' command in swagger-codegen (<= 2.2.2) and can lead to arbitrary code being executed when these commands are used on a well-crafted yaml specification.",3


## Decription processing

In [22]:
def similar(word, cpes) -> bool:
    for cpe in cpes:
        vendor, = cpe.vendor
        product, = cpe.product
        if word.lower() in {vendor.lower(), product.lower()}:
            return True
    
    return False

In [23]:
df.loc['FasterXML']

,cve_id,url,description,version_range,Java,JavaScript,Python
project,,,,,,,
jackson-databind,CVE-2017-15095,https://github.com/FasterXML/jackson-databind,A deserialization flaw was discovered in the j...,"(None, None)",6201368,0,0
jackson-databind,CVE-2017-17485,https://github.com/FasterXML/jackson-databind,FasterXML jackson-databind through 2.8.10 and ...,"(None, None)",6201368,0,0
jackson-databind,CVE-2017-7525,https://github.com/FasterXML/jackson-databind,A deserialization flaw was discovered in the j...,"(None, None)",6201368,0,0


In [24]:
descriptions, labels, cve_ids = list(zip(*toy_df[['description', 'label', 'cve_id']].values))

In [25]:
def create_feature_list_long(feature_extractor, sents, labels, cve_ids) -> list:
    """Uses all sentences to create feature list given feature extractor."""
    feature_list = list()
    for i, desc in enumerate(sents):
        label = labels[i]
        tagged_sent = nltk.pos_tag(nltk.word_tokenize(desc), tagset='universal')
        for j, (word, tag) in enumerate(tagged_sent):
            is_label = word == label
            features = feature_extractor(tagged_sent, j, cve_ids[i])
            feature_list.append((features, is_label))
    
    return feature_list

In [26]:
def create_feature_list_short(feature_extractor, sents, labels, cve_ids) -> list:
    """Uses only first sentence to create feature list given feature extractor."""
    feature_list = list()
    for i, desc in enumerate(sents):
        label = labels[i]
        first_sent = nltk.sent_tokenize(desc)[0]
        tagged_sent = nltk.pos_tag(nltk.word_tokenize(first_sent), tagset='universal')
        for j, (word, tag) in enumerate(tagged_sent):
            is_label = word == label
            features = feature_extractor(tagged_sent, j, cve_ids[i])
            feature_list.append((features, is_label))
    
    return feature_list

In [27]:
def create_feature_list_v2(feature_extractor, sents, labels, cve_ids) -> list:
    """Uses only first sentence to create feature list given feature extractor."""
    feature_list = list()
    for i, desc in enumerate(sents):
        label = labels[i]
        for j, sent in enumerate(nltk.sent_tokenize(desc)):
            tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent), tagset='universal')
            for k, (word, tag) in enumerate(tagged_sent):
                is_label = word == label
                features = feature_extractor(tagged_sent, sent_pos=j, word_pos=k, cve_id=cve_ids[i])
                feature_list.append((features, is_label))
    
    return feature_list

In [28]:
def eval_accuracy(classifier, extractor, sentences: list, labels: list, cve_ids: list, n=1) -> float:
    from collections import Counter
    
    assert len(sentences) == len(labels)
    
    accurates = [None] * len(labels)
    for i, sent in enumerate(sentences):
        tagged = nltk.pos_tag(nltk.word_tokenize(sent), tagset='universal')
        prob_dist = [classifier.prob_classify(extractor(tagged, j, cve_ids[i])) for j in range(len(tagged))]
        probs = [(word, tag, prob.prob(True)) for (word, tag), prob in zip(tagged, prob_dist)]
        probs  = sorted(probs, key=lambda x: x[2], reverse=True)
        
        most_prob = set([prob[0].lower() for prob in probs[:n]])
        accurates[i] = labels[i] in most_prob
    
#         if not accurates[i]:
#             print('Sentence: ', sent)
#             print('Expected: `%s`' % labels[i], 'got: `%s`' % probs, '\n')
    
    bag = Counter(accurates)
    return bag[True] / len(labels)

In [29]:
def eval_accuracy_v2(classifier, extractor, sentences: list, labels: list, cve_ids: list, n=1) -> float:
    from collections import Counter
    
    assert len(sentences) == len(labels)
    
    accurates = [None] * len(labels)
    for i, sent in enumerate(sentences):
        for s in nltk.sent_tokenize(sent):
            tagged = nltk.pos_tag(nltk.word_tokenize(s), tagset='universal')
            prob_dist = [classifier.prob_classify(extractor(tagged, s, j, cve_ids[i])) for j in range(len(tagged))]
            probs = [(word, tag, prob.prob(True)) for (word, tag), prob in zip(tagged, prob_dist)]
            probs  = sorted(probs, key=lambda x: x[2], reverse=True)

            most_prob = set([prob[0].lower() for prob in probs[:n]])
            accurates[i] = labels[i] in most_prob
    
#         print('Expected: `%s`' % labels[i], 'got: `%s`' % most_prob)
    
    # TODO: come up with more sophisticated way of measuring accuracy
    bag = Counter(accurates)
    return bag[True] / len(labels)

In [30]:
def extract_features_vanilla(tagged: list, pos: int, cve_id=None):
    """Extract contextual features from the sentence w.r.t given position of a word."""
    word, tag = tagged[pos]
    features = {
        'tag': tag,
        'has-uppercase': word[0].isupper(),
        'word-len': len(word) > 3,
    }
    if pos == 0:
        features['prev-tag'] = '<start>'
    else:
        features['prev-word'] = tagged[pos - 1][0].lower()
        features['prev-tag'] = tagged[pos - 1][1]
        
    return features

# Lets not split the dataset here for now
feature_list = create_feature_list_long(extract_features_vanilla, descriptions, labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

eval_accuracy(classifier, extract_features_vanilla, descriptions, labels, cve_ids, n=1)

Most Informative Features
               prev-word = 'the'           False : True   =      8.2 : 1.0
               prev-word = None            False : True   =      8.0 : 1.0
               prev-word = 'a'             False : True   =      7.7 : 1.0
                prev-tag = 'ADP'            True : False  =      5.0 : 1.0
                     tag = 'ADV'            True : False  =      5.0 : 1.0
                     tag = 'ADJ'            True : False  =      3.6 : 1.0
                prev-tag = 'NOUN'          False : True   =      2.5 : 1.0
               prev-word = 'aka'            True : False  =      2.3 : 1.0
                prev-tag = '<start>'        True : False  =      1.8 : 1.0
                word-len = True             True : False  =      1.7 : 1.0


0.15702479338842976

In [31]:
def extract_features_v0(tagged: list, pos: int, cve_id=None):
    """Extract contextual features from the sentence w.r.t given position of a word."""
    word, tag = tagged[pos]
    features = {
        'tag': tag,
        'vend_prod_match': similar(word, cves[cve_id].get_cpe(cpe_type='a')),
        'word-len': len(word) > 3,
    }
    if pos == 0:
        features['prev-tag'] = '<start>'
    else:
        if pos > 1:
            features['prev-tag'] = tagged[pos - 1][1]
            features['prev-bigram'] = " ".join(w.lower() for w, t in tagged[pos - 2: pos])
            
        features['prev-word'] = tagged[pos - 1][0].lower()
        features['prev-tag'] = tagged[pos - 1][1]
        
    return features

# Lets not split the dataset here for now
feature_list = create_feature_list_long(extract_features_v0, descriptions, labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

eval_accuracy(classifier, extract_features_v0, descriptions, labels, cve_ids, n=1)

Most Informative Features
         vend_prod_match = True             True : False  =     37.6 : 1.0
             prev-bigram = None            False : True   =     19.0 : 1.0
         vend_prod_match = False           False : True   =     10.9 : 1.0
               prev-word = 'the'           False : True   =      8.2 : 1.0
               prev-word = None            False : True   =      8.0 : 1.0
               prev-word = 'a'             False : True   =      7.7 : 1.0
             prev-bigram = 'is for'         True : False  =      5.1 : 1.0
                prev-tag = 'ADP'            True : False  =      5.0 : 1.0
                     tag = 'ADV'            True : False  =      5.0 : 1.0
             prev-bigram = 'in the'        False : True   =      4.5 : 1.0


0.8760330578512396

In [32]:
def extract_features_v1(tagged: list, pos: int, cve_id=None):
    """Extract contextual features from the sentence w.r.t given position of a word."""
    word, tag = tagged[pos]
    cases = [w.isupper() for w in word]
    features = {
        'tag': tag,
        'has-uppercase': any(cases) and not all(cases),
        'vend_prod_match': similar(word, cves[cve_id].get_cpe(cpe_type='a')),
        'word-len-threshold': len(word) > 3
    }
    if pos == 0:
        features['prev-tag'] = '<start>'
    else:
        if pos > 1:
            features['prev-tag'] = tagged[pos - 1][1]
            features['prev-bigram'] = " ".join(w.lower() for w, t in tagged[pos - 2: pos])
            
        features['prev-word'] = tagged[pos - 1][0].lower()
        features['prev-tag'] = tagged[pos - 1][1]
        
    return features

# Lets not split the dataset here for now
feature_list = create_feature_list_long(extract_features_v1, descriptions, labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

eval_accuracy(classifier, extract_features_v1, descriptions, labels, cve_ids, n=1)

Most Informative Features
         vend_prod_match = True             True : False  =     37.6 : 1.0
             prev-bigram = None            False : True   =     19.0 : 1.0
         vend_prod_match = False           False : True   =     10.9 : 1.0
               prev-word = 'the'           False : True   =      8.2 : 1.0
               prev-word = None            False : True   =      8.0 : 1.0
               prev-word = 'a'             False : True   =      7.7 : 1.0
             prev-bigram = 'is for'         True : False  =      5.1 : 1.0
                prev-tag = 'ADP'            True : False  =      5.0 : 1.0
                     tag = 'ADV'            True : False  =      5.0 : 1.0
             prev-bigram = 'in the'        False : True   =      4.5 : 1.0


0.8842975206611571

In [33]:
regex_tokenizer = nltk.RegexpTokenizer(pattern=u"[-_]", gaps=True)
# nltk.pos_tag(regex_tokenizer.tokenize(v), tagset='universal')

In [34]:
def extract_features_v2(tagged: list, pos: int, cve_id=None):
    """Extract contextual features from the sentence w.r.t given position of a word."""
    word, tag = tagged[pos]
    features = {
        'tag': tag,
        'word-len': len(word) > 3,
        'has-uppercase': any(w.isupper() for w in word),
        'vend_prod_match': similar(word, cves[cve_id].get_cpe(cpe_type='a')),
    }
    if pos == 0:
        features['prev-tag'] = '<start>'
    else:
        if pos > 1:
            features['prev-tag'] = tagged[pos - 1][1]
            features['prev-bigram'] = " ".join(w.lower() for w, t in tagged[pos - 2: pos])
            
        
        if pos < len(tagged):    
            features['next-bigram'] = " ".join(w.lower() for w, t in tagged[pos + 1: pos + 3])
            features['next-bigram-tags'] = " ".join(t for w, t in tagged[pos + 1: pos + 3])
        
    return features

# Lets not split the dataset here for now
feature_list = create_feature_list_long(extract_features_v2, descriptions, labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

eval_accuracy(classifier, extract_features_v2, descriptions, labels, cve_ids, n=1)

Most Informative Features
         vend_prod_match = True             True : False  =     37.6 : 1.0
             next-bigram = '3.0 .'          True : False  =     32.5 : 1.0
        next-bigram-tags = 'NUM ADJ'        True : False  =     27.3 : 1.0
             prev-bigram = None            False : True   =     19.0 : 1.0
             next-bigram = None            False : True   =     15.6 : 1.0
         vend_prod_match = False           False : True   =     10.9 : 1.0
        next-bigram-tags = 'NUM .'          True : False  =      9.4 : 1.0
        next-bigram-tags = 'NUM NOUN'       True : False  =      5.8 : 1.0
                prev-tag = 'ADP'            True : False  =      5.2 : 1.0
             prev-bigram = 'is for'         True : False  =      5.1 : 1.0


0.8512396694214877

In [35]:
def extract_features_v3(tagged: list, pos: int, cve_id=None):
    """Extract contextual features from the sentence w.r.t given position of a word."""
    word, tag = tagged[pos]
    try:
        ver_pos = [pos for pos, (w, t) in enumerate(tagged[pos:]) if t == 'NUM'][0]
    except:
        ver_pos = None
        
    ver_follows = any([pos for pos, (w, t) in enumerate(tagged[pos:]) if t == 'NUM' or re.search(u'version', w)])
    features = {
        'tag': tag,
        'word-len': len(word) > 3,
        'vend_prod_match': similar(word, cves[cve_id].get_cpe(cpe_type='a')),
        'version_pos': ver_pos,
        'ver_follows': ver_follows
    }
    if pos == 0:
        features['prev-tag'] = '<start>'
    else:
        if pos < len(tagged) - 1:    
            features['next-bigram'] = " ".join(w.lower() for w, t in tagged[pos + 1: pos + 3])
            features['next-bigram-tags'] = " ".join(t for w, t in tagged[pos + 1: pos + 3])
            
        features['prev-word'] = tagged[pos - 1][0]
            
            
    return features

# Lets not split the dataset here for now
feature_list = create_feature_list_long(extract_features_v3, descriptions, labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

eval_accuracy(classifier, extract_features_v3, descriptions, labels, cve_ids, n=2)

Most Informative Features
         vend_prod_match = True             True : False  =     37.6 : 1.0
             next-bigram = '3.0 .'          True : False  =     32.6 : 1.0
             next-bigram = None            False : True   =     31.3 : 1.0
        next-bigram-tags = 'NUM ADJ'        True : False  =     27.5 : 1.0
         vend_prod_match = False           False : True   =     10.9 : 1.0
        next-bigram-tags = 'NUM .'          True : False  =      9.5 : 1.0
               prev-word = None            False : True   =      8.4 : 1.0
             version_pos = None            False : True   =      8.0 : 1.0
               prev-word = 'the'           False : True   =      7.1 : 1.0
               prev-word = 'a'             False : True   =      7.0 : 1.0


0.9752066115702479

In [36]:
def extract_features_v4(tagged: list, sent_pos:int, word_pos: int, cve_id=None):
    """Extract contextual features from the sentence w.r.t given position of a word."""
    word, tag = tagged[word_pos]
    try:
        ver_pos = [pos for pos, (w, t) in enumerate(tagged[word_pos:]) if t == 'NUM'][0]
    except:
        ver_pos = None
    features = {
        'tag': tag,
        'word_len': len(word) > 3,
        'vend_prod_match': similar(word, cves[cve_id].get_cpe(cpe_type='a')),
        'version_pos': ver_pos,
        'sent_pos': sent_pos
    }
    if word_pos == 0:
        features['prev-tag'] = '<start>'
    else:
        if word_pos > 1:
            features['prev_tag'] = tagged[word_pos - 1][1]
            features['prev_bigram'] = " ".join(w.lower() for w, t in tagged[word_pos - 2: word_pos])
        
        if word_pos < len(tagged) - 1:    
            features['next_bigram'] = " ".join(w.lower() for w, t in tagged[word_pos + 1: word_pos + 3])
            features['next_bigram-tags'] = " ".join(t for w, t in tagged[word_pos + 1: word_pos + 3])
            
    return features

# Lets not split the dataset here for now
feature_list = create_feature_list_v2(extract_features_v4, descriptions, labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

eval_accuracy_v2(classifier, extract_features_v4, descriptions, labels, cve_ids, n=1)

Most Informative Features
             next_bigram = None            False : True   =     49.2 : 1.0
         vend_prod_match = True             True : False  =     37.6 : 1.0
             next_bigram = '3.0 .'          True : False  =     33.3 : 1.0
             prev_bigram = None            False : True   =     29.9 : 1.0
        next_bigram-tags = 'NUM ADJ'        True : False  =     27.7 : 1.0
         vend_prod_match = False           False : True   =     10.9 : 1.0
             version_pos = 1                True : False  =      9.9 : 1.0
        next_bigram-tags = 'NUM .'          True : False  =      9.5 : 1.0
        next_bigram-tags = 'NUM NOUN'       True : False  =      5.9 : 1.0
             version_pos = None            False : True   =      5.9 : 1.0


0.6115702479338843

## Test accuracy on the toy dataset using multiple feature extractors

In [37]:
x = np.arange(1, 15)
feature_extractors = [extract_features_vanilla, extract_features_v0, extract_features_v1, extract_features_v2,
                      extract_features_v3
                     ]

split = int(len(descriptions) * 0.2)
test_set, test_labels = descriptions[:split], labels[:split]
train_set, train_labels = descriptions[split:], labels[split:]

accuracy_list = list()
for i, extractor in enumerate(feature_extractors):
    # Lets not split the dataset here for now
    feature_list = create_feature_list_long(extractor, train_set, train_labels, cve_ids[split:])

    classifier = nltk.NaiveBayesClassifier.train(feature_list)
#     classifier.show_most_informative_features()
    
    accuracy_list.append([eval_accuracy(classifier, extractor, test_set, test_labels, cve_ids[:split], i) for i in x])
    
feature_list = create_feature_list_v2(extract_features_v4, train_set, train_labels, cve_ids[split:])

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

accuracy_list.append([eval_accuracy_v2(classifier, extract_features_v4, test_set, test_labels, cve_ids[:split], i) for i in x])

Most Informative Features
         vend_prod_match = True             True : False  =     45.1 : 1.0
             next_bigram = None            False : True   =     40.0 : 1.0
             next_bigram = '3.0 .'          True : False  =     34.2 : 1.0
        next_bigram-tags = 'NUM ADJ'        True : False  =     24.4 : 1.0
             prev_bigram = None            False : True   =     24.3 : 1.0
             version_pos = 1                True : False  =     13.9 : 1.0
        next_bigram-tags = 'NUM .'          True : False  =     12.7 : 1.0
         vend_prod_match = False           False : True   =     10.6 : 1.0
        next_bigram-tags = 'NUM CONJ'       True : False  =      8.1 : 1.0
        next_bigram-tags = 'NUM NOUN'       True : False  =      6.6 : 1.0


### Current approach

In [38]:
from collections import OrderedDict


def get_first_sentence(description):
    """Get only the first sentence from the description."""
    sentences = nltk.sent_tokenize(description)
    return sentences[0] if sentences else ''


def guess_package_name(description):
    from nltk.corpus import stopwords
    """Guess package name from given description.

    Very naive approach. Words starting with uppercase letter
    are considered to be possible package names (minus stop words).

    Returns a list of possible package names, without duplicates.
    """

    stop_words = set()

    try:
        # Fails when no downloaded stopwords are available.
        stop_words.update(stopwords.words('english'))
    except LookupError:
        # Download stopwords since they are not available.
        nltk.download('stopwords')
        stop_words.update(stopwords.words('english'))

    regexp = re.compile('[A-Z][A-Za-z0-9-:]*')  # ? TODO: tweak
    suspects = regexp.findall(description)

    results = []

    if not suspects:
        return results

    results = [x.lower() for x in suspects if x.lower() not in stop_words]
    # get rid of duplicates, but keep order
    results = list(OrderedDict.fromkeys(results))

    return results


def get_package_name_candidates(description):
    """Try to identify possible package names in the CVE's description."""
    pkg_name_candidates = set()
    first_sentence = get_first_sentence(description)
    names = guess_package_name(first_sentence)
    pkg_name_candidates.update(set(names))
    return pkg_name_candidates

In [39]:
def eval_old_accuracy(sentences: list, labels: list) -> float:
    from collections import Counter
    
    assert len(sentences) == len(labels)
    
    guesses = [None] * len(labels)
    accurates = [None] * len(labels)
    for i, sent in enumerate(sentences):
        
        names = get_package_name_candidates(sent)
        guesses[i] = len(names)
        accurates[i] = labels[i] in names # only taking the first word .. needs better implementation
    
#         print('Expected: `%s`' % labels[i], 'got: `%s`' % most_prob)
    
    # TODO: come up with more sophisticated way of measuring accuracy
    bag = Counter(accurates)
    return bag[True] / len(labels), sum(guesses) / len(guesses)

In [40]:
def get_nof_guesses(sentences: list, labels: list) -> float:
    assert len(sentences) == len(labels)
    
    guesses = [None] * len(labels)
    for i, sent in enumerate(sentences):
        
        names = get_package_name_candidates(sent)
        guesses[i] = len(names)
        
    # TODO: come up with more sophisticated way of measuring accuracy
    return guesses

### Evaluation of the current approach on the test sets

In [41]:
old_accuracy, mean_guess = eval_old_accuracy(test_set, test_labels)

### Evaluation of the current approach on the whole set

In [42]:
eval_old_accuracy(descriptions, labels)

(0.7355371900826446, 3.347107438016529)

## Draw extractor accuracy

In [43]:
trace_names = ['vanilla_extractor'] + ['extract_features_v%d' % i for i in range(len(accuracy_list))]
data = [go.Scatter(x=x, y=ac, name=trace_names[i]) for i, ac in enumerate(accuracy_list)]

layout = go.Layout(
    yaxis=dict(
        title='Accuracy',
        titlefont=dict(
            color='grey'
        )
    ),
    xaxis=dict(
        title='Candidates',
        titlefont=dict(
            color='grey'
        )
    ),
    shapes=[
        {
            'type': 'line',
            'x0': mean_guess,
            'x1': mean_guess,
            'y0': -0.05,
            'y1': 1.1,
            'opacity': 0.2,
            'line': {
                'dash': 'dash'
            }
        },
        {
            'type': 'line',
            'x0': -0.5,
            'x1': 20,
            'y0': old_accuracy,
            'y1': old_accuracy,
            'opacity': 0.2,
            'line': {
                'dash': 'dash'
            }
        }
    ]
)

fig = go.Figure(data=data, layout=layout)

In [44]:
iplot(fig, show_link=False)

In [45]:
guess_trace = go.Scatter(y=get_nof_guesses(test_set, test_labels))
layout = go.Layout(
    xaxis=dict(
        ticks='',
        showticklabels=False,
        showgrid=False
    ),
    shapes=[
        {
            'type': 'line',
            'y0': mean_guess,
            'y1': mean_guess,
            'x0': -0.1,
            'x1': 23,
            'opacity': 0.2,
            'line': {
                'dash': 'dash'
            }
        },
    ]
)

fig = go.Figure(data=[guess_trace], layout=layout)

In [46]:
iplot(fig, show_link=False)

## Remove words/stopwords when evaluating accuracy

In [47]:
stopwords = set(nltk.corpus.stopwords.words())

def eval_accuracy_spec(classifier, extractor, sentences: list, labels: list, cve_ids: list, n=1) -> float:
    from collections import Counter
    
    assert len(sentences) == len(labels)
    
    accurates = [None] * len(labels)
    for i, sent in enumerate(sentences):
        tagged = nltk.pos_tag(nltk.word_tokenize(sent), tagset='universal')
        prob_dist = [classifier.prob_classify(extractor(tagged, j, cve_ids[i])) for j in range(len(tagged))]
        # get probs and remove stopwords
        probs = [(word, tag, prob.prob(True)) for (word, tag), prob in zip(tagged, prob_dist) if word not in stopwords and tag == 'NOUN']
        print(probs)
        probs = sorted(probs, key=lambda x: x[2], reverse=True)
        
        most_prob = set([prob[0].lower() for prob in probs[:n]])
        accurates[i] = labels[i] in most_prob
    
        if not accurates[i]:
            print('Sentence: ', sent)
            print('Expected: `%s`' % labels[i], 'got: `%s`' % most_prob)
    
    bag = Counter(accurates)
    return bag[True] / len(labels)

In [48]:
accuracy_list = list()
for i, extractor in enumerate(feature_extractors):
    # Lets not split the dataset here for now
    feature_list = create_feature_list_long(extractor, train_set, train_labels, cve_ids[split:])

    classifier = nltk.NaiveBayesClassifier.train(feature_list)
#     classifier.show_most_informative_features()
    
    accuracy_list.append([eval_accuracy_spec(classifier, extractor, test_set, test_labels, cve_ids[:split], i) for i in x])
    
feature_list = create_feature_list_v2(extract_features_v4, train_set, train_labels, cve_ids)

classifier = nltk.NaiveBayesClassifier.train(feature_list)
classifier.show_most_informative_features()

accuracy_list.append([eval_accuracy_v2(classifier, extract_features_v4, test_set, test_labels, cve_ids, i) for i in x])

[('MODX', 'NOUN', 0.0030510463144199), ('Revolution', 'NOUN', 0.005122980583748046), ('version', 'NOUN', 0.036648633813404635), ('SQL', 'NOUN', 4.9624460059082774e-05), ('injection', 'NOUN', 0.036648633813404635), ('sanitization', 'NOUN', 0.019112587043665932), ('escape', 'NOUN', 0.0018203790518677395), ('method', 'NOUN', 0.036648633813404635), ('user', 'NOUN', 0.001352519117281349), ('database', 'NOUN', 0.014679171686772737), ('privileges', 'NOUN', 0.014679171686772737)]
Sentence:  MODX Revolution version 2.x - 2.5.6 is vulnerable to blind SQL injection caused by improper sanitization by the escape method resulting in authenticated user accessing database and possibly escalating privileges.
Expected: `revolution` got: `{'version'}`
[('MODX', 'NOUN', 0.007989493693372844), ('Revolution', 'NOUN', 0.005122980583748046), ('name', 'NOUN', 0.0004506386520118372), ('parameters', 'NOUN', 0.0004749859279574337), ('System', 'NOUN', 0.00024678921490626734), ('Settings', 'NOUN', 0.005122980583748

[('MODX', 'NOUN', 0.007989493693372844), ('Revolution', 'NOUN', 0.005122980583748046), ('user', 'NOUN', 0.0019043770103975462), ('file', 'NOUN', 0.02778263840425215), ('permissions', 'NOUN', 0.019112587043665932), ('code', 'NOUN', 0.0004750180486773723), ('file', 'NOUN', 0.0019043770103975462), ('name', 'NOUN', 0.0018203790518677395), ('.htaccess', 'NOUN', 0.005405316957769816)]
Sentence:  In MODX Revolution before 2.5.7, a user with file upload permissions is able to execute arbitrary code by uploading a file with the name .htaccess.
Expected: `revolution` got: `{'permissions', 'file'}`
[('MODX', 'NOUN', 0.007989493693372844), ('Revolution', 'NOUN', 0.005122980583748046), ('user', 'NOUN', 0.0019043770103975462), ('resource', 'NOUN', 0.02778263840425215), ('edit', 'NOUN', 0.036648633813404635), ('permissions', 'NOUN', 0.036648633813404635), ('payload', 'NOUN', 0.0004750180486773723), ('title', 'NOUN', 0.0018203790518677395), ('post', 'NOUN', 0.013867726364218418), ('parameter', 'NOUN',

Expected: `revolution` got: `{'conjunction', 'header', 'modx'}`
[('SimpleXML', 'NOUN', 0.0030510463144199), ('version', 'NOUN', 0.019112587043665932), ('XXE', 'NOUN', 3.077401469962619e-06), ('vulnerability', 'NOUN', 0.012522157924691966), ('SSRF', 'NOUN', 0.0020124691908496566), ('information', 'NOUN', 8.412130067504175e-05), ('disclosure', 'NOUN', 0.012522157924691966), ('DoS', 'NOUN', 2.8024369610121127e-07)]
Sentence:  SimpleXML (latest version 2.7.1) is vulnerable to an XXE vulnerability resulting SSRF, information disclosure, DoS and so on.
Expected: `simplexml` got: `{'disclosure', 'version', 'vulnerability'}`
[('vulnerability', 'NOUN', 0.0019043770103975462), ('Swagger-Parser', 'NOUN', 0.007989493693372844), ('version', 'NOUN', 0.008125093331013947), ('<', 'NOUN', 0.0009353623407946211), ('Swagger', 'NOUN', 0.0001090177181564065), ('version', 'NOUN', 0.014679171686772737), ('<', 'NOUN', 0.0009353623407946211), ('yaml', 'NOUN', 0.05145119843070482), ('functionality', 'NOUN', 0.0

Expected: `nv-websocket-client` got: `{'servers', 'certificate', 'client', 'name'}`
[('ELabftw', 'NOUN', 0.0030510463144199), ('version', 'NOUN', 0.036648633813404635), ('scripting', 'NOUN', 0.0007787928378122423), ('experiment', 'NOUN', 0.0018203790518677395), ('infos', 'NOUN', 0.036648633813404635), ('component', 'NOUN', 0.036648633813404635), ('execution', 'NOUN', 0.0004750180486773723), ('JavaScript', 'NOUN', 0.0005376493676017715), ('denial', 'NOUN', 0.0008048491969533984), ('service', 'NOUN', 0.00395849315498599)]
Sentence:  ELabftw version 1.7.8 is vulnerable to stored cross-site scripting in the experiment infos component resulting in arbitrary execution of JavaScript and denial of service.
Expected: `elabftw` got: `{'version', 'infos', 'service', 'component'}`
[('Primetek', 'NOUN', 0.0030510463144199), ('Primefaces', 'NOUN', 0.005122980583748046), ('encryption', 'NOUN', 0.006453085790469036), ('flaw', 'NOUN', 0.012522157924691966), ('code', 'NOUN', 0.00023470388573745442), ('e

[('Odoo', 'NOUN', 0.007989493693372844), ('Odoo', 'NOUN', 1.1387222515489144e-05), ('Community', 'NOUN', 0.005122980583748046), ('Edition', 'NOUN', 0.0017135123883797992), ('Odoo', 'NOUN', 0.0001090177181564065), ('Enterprise', 'NOUN', 0.005122980583748046), ('Edition', 'NOUN', 0.005122980583748046), ('attackers', 'NOUN', 0.00023470388573745442), ('authentication', 'NOUN', 0.002970720510479935), ('circumstances', 'NOUN', 0.0038818714941932647), ('parameters', 'NOUN', 0.051732965788054946), ('characters', 'NOUN', 0.05145119843070482), ('database', 'NOUN', 0.0018203790518677395), ('layer', 'NOUN', 0.036648633813404635), ('Psycopg', 'NOUN', 0.0073302996519779085)]
[('Odoo', 'NOUN', 0.007989493693372844), ('Odoo', 'NOUN', 1.1387222515489144e-05), ('Community', 'NOUN', 0.005122980583748046), ('Edition', 'NOUN', 0.0017135123883797992), ('Odoo', 'NOUN', 0.0001090177181564065), ('Enterprise', 'NOUN', 0.005122980583748046), ('Edition', 'NOUN', 0.005122980583748046), ('access', 'NOUN', 0.0017682

...

[('Java', 'NOUN', 2.215129492197324e-06), ('WebSocket', 'NOUN', 5.242209922260807e-05), ('client', 'NOUN', 0.00035092420514156754), ('nv-websocket-client', 'NOUN', 0.009526114535855678), ('server', 'NOUN', 6.336555446547773e-06), ('hostname', 'NOUN', 2.861914407639645e-05), ('domain', 'NOUN', 2.413285920556187e-06), ('name', 'NOUN', 2.854231549119261e-06), ('subject', 'NOUN', 0.0001001081878074313), ('Common', 'NOUN', 0.0001534472632015623), ('Name', 'NOUN', 0.0009702433116337953), ('CN', 'NOUN', 2.4475716220341628e-08), ('field', 'NOUN', 8.399549123135223e-06), ('X.509', 'NOUN', 1.9391242928082182e-05), ('certificate', 'NOUN', 9.278182331206033e-06), ('attackers', 'NOUN', 0.0006159036355421464), ('SSL/TLS', 'NOUN', 0.0001407555368534055), ('servers', 'NOUN', 9.798596069908157e-05), ('certificate', 'NOUN', 2.210656276593673e-06)]
[('ELabftw', 'NOUN', 0.8864184851585389), ('version', 'NOUN', 0.5714050164356131), ('scripting', 'NOUN', 9.514123267404741e-07), ('experiment', 'NOUN', 2.8388

[('Odoo', 'NOUN', 0.9966825814086888), ('Odoo', 'NOUN', 0.024295547738791314), ('Community', 'NOUN', 0.3033919865071487), ('Edition', 'NOUN', 0.7834888393006215), ('Odoo', 'NOUN', 0.12733293682804228), ('Enterprise', 'NOUN', 0.3033919865071487), ('Edition', 'NOUN', 0.9156552226751903), ('access', 'NOUN', 1.5641461092381227e-05), ('control', 'NOUN', 9.464204646333983e-05), ('OAuth', 'NOUN', 2.8154277667055393e-05), ('tokens', 'NOUN', 2.854231549119261e-06), ('OAuth', 'NOUN', 6.336555446547773e-06), ('module', 'NOUN', 2.7038018647034964e-05), ('users', 'NOUN', 5.60226073470477e-05), ('OAuth', 'NOUN', 0.0001407555368534055), ('sessions', 'NOUN', 0.0008230668829909114), ('users', 'NOUN', 2.009691563028131e-07)]
[('Directory', 'NOUN', 0.003620451670542673), ('traversal', 'NOUN', 0.0022861083924746012), ('vulnerability', 'NOUN', 0.03893160445107227), ('tools.file_open', 'NOUN', 0.018601042156985582), ('Odoo', 'NOUN', 0.9988495720146102), ('allows', 'NOUN', 9.192329838266498e-05), ('users', '

[('vulnerability', 'NOUN', 0.0020789720050082617), ('Swagger-Parser', 'NOUN', 0.9857584236232351), ('version', 'NOUN', 0.0006854085734517694), ('<', 'NOUN', 0.010604707649251905), ('=', 'NOUN', 0.12230447475664998), ('yaml', 'NOUN', 0.0016869754884054574), ('functionality', 'NOUN', 0.06391769411946426), ('results', 'NOUN', 0.10515763059902429), ('code', 'NOUN', 0.004775244577621033), ('specification', 'NOUN', 0.13737561892184097), ("'generate", 'NOUN', 0.1367054820086479), ("'validate", 'NOUN', 0.0031592905637227416), ('command', 'NOUN', 0.003767222836249654), ('swagger-codegen', 'NOUN', 0.9574839300441667), ('=', 'NOUN', 0.12230447475664998), ('code', 'NOUN', 1.1211063219465053e-05), ('commands', 'NOUN', 0.00015319610911000917), ('yaml', 'NOUN', 0.00107314281744077), ('specification', 'NOUN', 2.210656276593673e-06)]
[('Java', 'NOUN', 2.215129492197324e-06), ('WebSocket', 'NOUN', 5.242209922260807e-05), ('client', 'NOUN', 0.00035092420514156754), ('nv-websocket-client', 'NOUN', 0.00952

[('MODX', 'NOUN', 0.8683135290829723), ('Revolution', 'NOUN', 0.9952149400277436), ('version', 'NOUN', 0.9444030599552373), ('SQL', 'NOUN', 8.639233761843758e-06), ('injection', 'NOUN', 0.0006251595543015433), ('sanitization', 'NOUN', 2.2613849268823193e-05), ('escape', 'NOUN', 6.336555446547773e-06), ('method', 'NOUN', 0.0006251595543015433), ('user', 'NOUN', 4.5288222641374016e-05), ('database', 'NOUN', 0.0008938050525954044), ('privileges', 'NOUN', 2.210656276593673e-06)]
[('MODX', 'NOUN', 0.8850984604482687), ('Revolution', 'NOUN', 0.9444030599552373), ('name', 'NOUN', 2.490157471573966e-05), ('parameters', 'NOUN', 7.714160744863847e-08), ('System', 'NOUN', 2.8388825964896435e-06), ('Settings', 'NOUN', 0.00035092420514156754), ('module', 'NOUN', 0.0005981903921491989), ('XSS', 'NOUN', 2.0874205710364934e-08), ('A', 'NOUN', 3.082819199009126e-08), ('payload', 'NOUN', 0.00020043748637982355), ('user', 'NOUN', 0.007506698982012778), ('module', 'NOUN', 3.158086379228234e-07)]
[('Direct

[('Odoo', 'NOUN', 0.9966825814086888), ('Odoo', 'NOUN', 0.024295547738791314), ('Community', 'NOUN', 0.3033919865071487), ('Edition', 'NOUN', 0.7834888393006215), ('Odoo', 'NOUN', 0.12733293682804228), ('Enterprise', 'NOUN', 0.3033919865071487), ('Edition', 'NOUN', 0.9156552226751903), ('attackers', 'NOUN', 0.0005359184085893051), ('authentication', 'NOUN', 0.008931949512865793), ('circumstances', 'NOUN', 0.050542353944247484), ('parameters', 'NOUN', 0.21001167748716784), ('characters', 'NOUN', 0.10339731584235361), ('database', 'NOUN', 0.0010596774007850269), ('layer', 'NOUN', 0.0010560154835249954), ('Psycopg', 'NOUN', 0.24753908758521723)]
[('Odoo', 'NOUN', 0.9966825814086888), ('Odoo', 'NOUN', 0.024295547738791314), ('Community', 'NOUN', 0.3033919865071487), ('Edition', 'NOUN', 0.7834888393006215), ('Odoo', 'NOUN', 0.12733293682804228), ('Enterprise', 'NOUN', 0.3033919865071487), ('Edition', 'NOUN', 0.9156552226751903), ('access', 'NOUN', 1.5641461092381227e-05), ('control', 'NOUN'

In [49]:
trace_names = ['vanilla_extractor'] + ['extract_features_v%d' % i for i in range(len(accuracy_list))]
data = [go.Scatter(x=x, y=ac, name=trace_names[i]) for i, ac in enumerate(accuracy_list)]

layout = go.Layout(
    yaxis=dict(
        title='Accuracy',
        titlefont=dict(
            color='grey'
        )
    ),
    xaxis=dict(
        title='Candidates',
        titlefont=dict(
            color='grey'
        )
    ),
    shapes=[
        {
            'type': 'line',
            'x0': mean_guess,
            'x1': mean_guess,
            'y0': -0.05,
            'y1': 1.1,
            'opacity': 0.2,
            'line': {
                'dash': 'dash'
            }
        },
        {
            'type': 'line',
            'x0': -0.5,
            'x1': 20,
            'y0': old_accuracy,
            'y1': old_accuracy,
            'opacity': 0.2,
            'line': {
                'dash': 'dash'
            }
        }
    ]
)

fig = go.Figure(data=data, layout=layout)

In [50]:
iplot(fig, show_link=False)

## Try predictions on df

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1125 entries, (Microsoft, ChakraCore) to (memcached, memcached)
Data columns (total 7 columns):
cve_id           1125 non-null object
url              1125 non-null object
description      1125 non-null object
version_range    1125 non-null object
Java             1125 non-null int64
JavaScript       1125 non-null int64
Python           1125 non-null int64
dtypes: int64(3), object(4)
memory usage: 81.9+ KB


In [52]:
def predict(sent:str, classifier=None, extractor=None, cve_id: list = None, n=1) -> float:
    tagged = nltk.pos_tag(nltk.word_tokenize(sent), tagset='universal')
    
    prob_dist = [classifier.prob_classify(extractor(tagged, j, cve_id)) for j in range(len(tagged))]
    
    probs = [(word, tag, prob.prob(True)) for (word, tag), prob in zip(tagged, prob_dist) if word not in stopwords and tag == 'NOUN' and not re.match(u"version[s]?", word)]
    probs = sorted(probs, key=lambda x: x[2], reverse=True)

    return probs[:n]

In [53]:
# Choose a classifier and train it on the whole df
feature_list = create_feature_list_long(extract_features_v3, descriptions, labels, cve_ids)
classifier = nltk.NaiveBayesClassifier.train(feature_list)

In [54]:
predictions = [None] * len(df)
for i, desc in enumerate(df.description.values):
    probs = predict(desc, classifier, extract_features_v3, cve_id=df.cve_id.values[i], n=3)
    predictions[i] = probs

In [55]:
# get just the names
pred_proj_names = [tuple(zip(*p))[0] for p in predictions]

In [56]:
pred_df = pd.Series(pred_proj_names, name='prediction')

In [57]:
df['prediction'] = pred_proj_names
df[(df.Java == 0) | (df.JavaScript == 0)][['description', 'prediction']].reset_index().style

,username,project,description,prediction
0,Microsoft,ChakraCore,"A remote code execution vulnerability exists when Microsoft scripting engine improperly accesses objects in memory. The vulnerability could corrupt memory in a way that enables an attacker to execute arbitrary code in the context of the current user. An attacker who successfully exploited the vulnerability could gain the same user rights as the current user, aka ""Scripting Engine Memory Corruption Vulnerability.""","('Microsoft', 'attacker', 'Vulnerability')"
1,Microsoft,ChakraCore,"A remote code execution vulnerability exists in the way affected Microsoft scripting engine render when handling objects in memory in Microsoft browsers. The vulnerability could corrupt memory in such a way that an attacker could execute arbitrary code in the context of the current user. An attacker who successfully exploited the vulnerability could gain the same user rights as the current user, aka ""Scripting Engine Memory Corruption Vulnerability.""","('Microsoft', 'Microsoft', 'attacker')"
2,Microsoft,ChakraCore,"An information disclosure vulnerability in Microsoft scripting engine allows remote attackers to obtain sensitive information from process memory via a crafted web site, aka ""Microsoft Browser Information Disclosure Vulnerability.""","('Microsoft', 'Microsoft', 'Disclosure')"
3,Microsoft,ChakraCore,"A remote code execution vulnerability exists in Microsoft Chakra Core in the way JavaScript engines render when handling objects in memory. aka ""Scripting Engine Memory Corruption Vulnerability"". This vulnerability is unique from CVE-2017-0252.","('Microsoft', 'Vulnerability', 'render')"
4,Microsoft,ChakraCore,"A remote code execution vulnerability exists in Microsoft Chakra Core in the way JavaScript engines render when handling objects in memory. aka ""Scripting Engine Memory Corruption Vulnerability"". This vulnerability is unique from CVE-2017-0223.","('Microsoft', 'Vulnerability', 'render')"
5,torproject,tor,The hidden-service feature in Tor before 0.3.0.8 allows a denial of service (assertion failure and daemon exit) in the relay_send_end_cell_from_edge_ function via a malformed BEGIN cell.,"('Tor', 'feature', 'BEGIN')"
6,torproject,tor,The hidden-service feature in Tor before 0.3.0.8 allows a denial of service (assertion failure and daemon exit) in the connection_edge_process_relay_cell function via a BEGIN_DIR cell on a rendezvous circuit.,"('Tor', 'feature', 'failure')"
7,torproject,tor,"Tor 0.3.x before 0.3.0.9 has a guard-selection algorithm that only considers the exit relay (not the exit relay's family), which might allow remote attackers to defeat intended anonymity properties by leveraging the existence of large families.","('Tor', 'relay', 'algorithm')"
8,torproject,tor,"The rend_service_intro_established function in or/rendservice.c in Tor before 0.2.8.15, 0.2.9.x before 0.2.9.12, 0.3.0.x before 0.3.0.11, 0.3.1.x before 0.3.1.7, and 0.3.2.x before 0.3.2.1-alpha, when SafeLogging is disabled, allows attackers to obtain sensitive information by leveraging access to the log files of a hidden service, because uninitialized stack data is included in an error message about construction of an introduction point circuit.","('Tor', 'function', 'or/rendservice.c')"
9,lota,phamm,"XSS exists in the login_form function in views/helpers.php in Phamm before 0.6.7, exploitable via the PATH_INFO to main.php.","('Phamm', 'function', 'views/helpers.php')"
